In [ ]:
import os
import nltk
import json
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/MyDrive/NLP_assignment2_data')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/NLP_assignment2_data'

In [3]:
train=open("train.txt","r")
train=train.read()
train=train.lower()

sentences=train.split('\n')     #considering a sentence a full line until \n
vocabulary=[]
unigram_count={}                #storing the words and their frequency
bigram_count={}                 #storing bigrams as tuples for example I am is stored as ('i','am'), and their frequency

In [4]:
print(len(sentences))

2206502


In [5]:
fraction_used=1                 #the fraction of total sentences being used
index=int(fraction_used*len(sentences))

In [6]:
sentences_using=sentences[:index]

for i in range(len(sentences_using)):
  words=nltk.word_tokenize(sentences_using[i])
  if (i%50000==0):
    print("Processed "+str(i))
  for j in words:
    if j in unigram_count:
      unigram_count[j]+=1
    else:
      unigram_count[j]=1

  for j in range(len(words)-1):
    if (words[j],words[j+1]) in bigram_count:
      bigram_count[(words[j],words[j+1])]+=1
    else:
      bigram_count[(words[j],words[j+1])]=1


Processed 0
Processed 50000
Processed 100000
Processed 150000
Processed 200000
Processed 250000
Processed 300000
Processed 350000
Processed 400000
Processed 450000
Processed 500000
Processed 550000
Processed 600000
Processed 650000
Processed 700000
Processed 750000
Processed 800000
Processed 850000
Processed 900000
Processed 950000
Processed 1000000
Processed 1050000
Processed 1100000
Processed 1150000
Processed 1200000
Processed 1250000
Processed 1300000
Processed 1350000
Processed 1400000
Processed 1450000
Processed 1500000
Processed 1550000
Processed 1600000
Processed 1650000
Processed 1700000
Processed 1750000
Processed 1800000
Processed 1850000
Processed 1900000
Processed 1950000
Processed 2000000
Processed 2050000
Processed 2100000
Processed 2150000
Processed 2200000


In [7]:
for i in unigram_count:
  vocabulary.append(i)        #adding all words in vocabulary

In [9]:
validation=[]                 #list of all questions is stored as list of dictionary
with open('validation.json') as f:
  for i in f:
    validation.append(json.loads(i))        

In [27]:
file1 = open("without_bonus_output.txt","a")

In [28]:
def without_bonus_smoothing(type_smoothing,validation):
  """ 
      given a question -> word1 word2 xxxxx word3
      type_smoothing=1 is for no smoothing
      predicting the bigram probability of (xxxxx|word2) = bigram_count(word2 xxxxx)/unigram_count(word2)

      type_smoothing=2 is for add 1 smoothing
      predicting the bigram probability of (xxxxx|word2) = (1+bigram_count(word2 xxxxx))/(unigram_count(word2)+length_vocabulary)

      type_smoothing=1 is for add k smoothing
      predicting the bigram probability of (xxxxx|word2) = (k+bigram_count(word2 xxxxx))/(unigram_count(word2)+k*length_vocabulary)

  """
  correct=0
  k=100    #for add k smoothing

  if(type_smoothing==1):
    file1.write("without smoothing \n")
  elif(type_smoothing==2):
    file1.write("add 1 smoothing \n")
  else:
    file1.write("add k smoothing \n")

  
  for i in validation:
    question=nltk.word_tokenize(i['question'].lower())
    xxxxx_index=question.index('xxxxx')           
    before=''   

    if(xxxxx_index==0):
      continue
    
    before=question[xxxxx_index-1]

    prob=0
    optimal_ans=''

    if(len(before)!=0):
      for j in i['options']:
        j=j.lower()
        to_check=(before,j)

        if(to_check not in bigram_count):
            bigram_count[to_check]=0

        if(type_smoothing!=1):
          if(before not in unigram_count):      #in case of no smoothing division by 0 error can occur if the case is not handled
            unigram_count[before]=0

        
        if(type_smoothing==1 and (before in unigram_count)):
          if(bigram_count[to_check]/unigram_count[before]>prob):      #considering the one with maxmimum probability
            prob=bigram_count[to_check]/unigram_count[before]         #without smoothing
            optimal_ans=j

        elif(type_smoothing==2):
          if((bigram_count[to_check]+1)/(unigram_count[before]+len(vocabulary))>prob):    #probability with add 1 smoothing
            prob=(bigram_count[to_check]+1)/(unigram_count[before]+len(vocabulary))
            optimal_ans=j

        elif(type_smoothing==3):
          if((bigram_count[to_check]+k)/(unigram_count[before]+k*len(vocabulary))>prob):   #probability with add k smoothing
            prob=(bigram_count[to_check]+k)/(unigram_count[before]+k*len(vocabulary))
            optimal_ans=j
    
    if(optimal_ans==i['answer']):
      correct+=1

    output="correct ans : "+i['answer'].lower()+", prediction is: "+optimal_ans + "\n"
    file1.write(output)

  print("Accuracy with type"+str(type_smoothing)+" smoothing is "+str(100*correct/len(validation))+" %")

In [29]:
without_bonus_smoothing(1,validation)
without_bonus_smoothing(2,validation)
without_bonus_smoothing(3,validation)

Accuracy with type1 smoothing is 51.2 %
Accuracy with type2 smoothing is 51.2 %
Accuracy with type3 smoothing is 51.2 %


In [30]:
file2 = open("with_bonus_output.txt","a")

In [31]:
def with_bonus_smoothing(type_smoothing,validation):
  """ 
      given a question -> word1 word2 xxxxx word3
      predicting using probability of P(xxxxx|word2) * P(word3|xxxxx) using the future context
      
      probability calculation is same as the previous part
      type_smoothing=1 is for no smoothing
      type_smoothing=2 is for add 1 smoothing
      type_smoothing=1 is for add k smoothing

  """

  if(type_smoothing==1):
    file2.write("without smoothing \n")
  elif(type_smoothing==2):
    file2.write("add 1 smoothing \n")
  else:
    file2.write("add k smoothing \n")

  correct=0
  k=100    #for add k smoothing
  for i in validation:
    question=nltk.word_tokenize(i['question'].lower())
    xxxxx_index=question.index('xxxxx')
    before=''
    after=''

    if(xxxxx_index!=0):
      before=question[xxxxx_index-1]
    
    if(xxxxx_index!=len(question)-1):
      after=question[xxxxx_index+1]

    prob=0
    optimal_ans=''
    
    if(len(before)!=0 and len(after)!=0):
      for j in i['options']:
        j=j.lower()
        to_check1=(before,j)
        to_check2=(j,after)
        
        if(to_check1 not in bigram_count):
            bigram_count[to_check1]=0

        if(to_check2 not in bigram_count):
            bigram_count[to_check2]=0

        if(type_smoothing!=1):
          if(before not in unigram_count):
            unigram_count[before]=0
            
          if(j not in unigram_count):
            unigram_count[before]=0

        bc_tocheck1=bigram_count[to_check1]
        bc_tocheck2=bigram_count[to_check2]
        uc_before=unigram_count[before]
        uc_j=unigram_count[j]
        len_vocab=len(vocabulary)

        if(type_smoothing==1 and (j in unigram_count and before in unigram_count)):
          if((bc_tocheck1/uc_before)*(bc_tocheck2/uc_j)>prob):
            prob=(bc_tocheck1/uc_before)*(bc_tocheck2/uc_j)
            optimal_ans=j

        elif(type_smoothing==2):
          if(((bc_tocheck1+1)/(uc_before+len_vocab))*((bc_tocheck2+1)/(uc_j+len_vocab))>prob):
            prob=((bc_tocheck1+1)/(uc_before+len_vocab))*((bc_tocheck2+1)/(uc_j+len_vocab))
            optimal_ans=j

        elif(type_smoothing==3):
          if(((bc_tocheck1+1)/(uc_before+k*len_vocab))*((bc_tocheck2+1)/(uc_j+k*len_vocab))>prob):
            prob=((bc_tocheck1+1)/(uc_before+k*len_vocab))*((bc_tocheck2+1)/(uc_j+k*len_vocab))
            optimal_ans=j

    if(optimal_ans==i['answer']):
      correct+=1

    output="correct ans : "+i['answer'].lower()+", prediction is: "+optimal_ans + "\n"
    file2.write(output)

  print("Accuracy with type"+str(type_smoothing)+" smoothing is "+str(100*correct/len(validation))+" %")

In [32]:
with_bonus_smoothing(1,validation)
with_bonus_smoothing(2,validation)
with_bonus_smoothing(3,validation)

Accuracy with type1 smoothing is 70.0 %
Accuracy with type2 smoothing is 64.7 %
Accuracy with type3 smoothing is 62.2 %
